In [4]:
# Required pkg
import Pkg;
Pkg.add("Images")
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("Interpolations")
Pkg.add("DataFrames")
Pkg.add("XLSX")
using Images, Statistics, LinearAlgebra, Plots, Interpolations, Random, DataFrames, XLSX

   Resolving package versions...
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\s2kjn\.julia\environments\v1.9\Project.toml`
  No Changes to 

### [Auxiliary functions] to get: training_images, testing_images
#### Each with a size = M * N^2

M = Number of images (min_images_per_folder)

N^2 = A vector of the original image_size*image_size image

In [5]:
# Get useful folders

# Return a Vector{String} with all the folders with {min_images_per_folder} inside.
function get_useful_folders(min_images)
    ### Getting images
    useful_folders = Vector{String}()
    
    itr = walkdir(base_dir)    
    (root, dirs, files) = first(itr)
    for d in dirs
        images_on_folder = readdir(base_dir * d)
        if size(images_on_folder)[1] >= min_images
            push!(useful_folders, base_dir * d * "/")
        end
    end
    useful_folders
end

# After filter by get_useful_folders(), get random {n_folders}
function get_random_n_folders(min_images_per_folder)
    # Get all the useful folders
    useful_folders = get_useful_folders(min_images_per_folder)
    # Get random folders from {useful_folders}
    rand(useful_folders, n_folders)
end

# Get all images from {n_folders} random folders, each case for {training_images} and {testing_images}
# Return a Matrix, size = M * N^2
#     M = Number of images (min_images_per_folder)
#     N^2 = A row vector of the original N*N image (now: 1*N^2)

function get_images(float_type, working_with=nothing)
    
    # Pick {n_folders} to work with if {working_with} not provided
    if working_with === nothing
       working_with = get_random_n_folders(min_images_per_folder)
    end    
        
    training_images = zeros(float_type, size(working_with, 1)*n_training_images,  image_size*image_size)
    testing_images = zeros(float_type, size(working_with, 1)*n_testing_images,  image_size*image_size)
    
    # For every folder, get pictures
    for (i, folder) in enumerate(working_with)
        cdir = readdir(folder)        
        
        # Google Drive create a "desktop.ini", then I have to "findfirst" and delete element
        splice!(cdir, findfirst(contains("desktop.ini"), cdir))
            
        training_count = 1
        testing_count = 1

        for image_relative_path in cdir 
            
            row_im = vec(float_type.(Gray.( imresize(load(folder * image_relative_path), image_size, image_size) )))' # 1 x N*N
            #display(Gray.(load(folder * image_relative_path)))
            
            if training_count <= n_training_images
                training_images[ (n_training_images*(i-1) + training_count),:] = row_im
                training_count += 1
            elseif testing_count <= n_testing_images
                testing_images[ (n_testing_images*(i-1) + testing_count),:] = row_im
                testing_count += 1
            else
                break  # Stop here if both training and testing images are collected
            end
        end
    end
    (training_images, testing_images)
end

get_images (generic function with 2 methods)

In [6]:
function show_training_images(images)
    for i in 1:n_folders
        row = vcat([ Gray.(reshape(images[j,:], image_size, image_size)) for j in (n_training_images*(i-1))+1:((n_training_images*(i-1))+n_training_images)])
        display(row)
    end    
end

function show_testing_images(images)
    for i in 1:n_folders
        row = vcat([ Gray.(reshape(images[j,:], image_size, image_size)) for j in (n_testing_images*(i-1))+1:((n_testing_images*(i-1))+n_testing_images)])
        display(row)
    end    
end

show_testing_images (generic function with 1 method)

# Getting the basics

### μ, A, C, eigenvalues, eigenvectors

#### n_folders
Number of folders to use, randomly selected. **Between 1 and _size(get_useful_folders(n_folders), 1)_**

#### n_training_images
Number of training images to pick from every folder

#### n_testing_images
Same but for testing

#### min_images_per_folder
Before, it will filter the folders on the directory by {min_images_per_folder}

In [7]:
# ./lfw/  = Color, not aligned
# ./lfw2/ = Black and white, already aligned (eyes)
base_dir = "./lfw/" 
image_size = 64

n_folders = 10
n_training_images = 12
n_testing_images = 6

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

#### (training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used

In [8]:
function setup_basics(float_type, folders=nothing)
    (training_images, testing_images) = get_images(float_type, folders)
    
    # Mean
    μ = mean(training_images, dims = 1)

    # Centered matrix
    A = training_images.-μ

    # Covariance (N2xN2) and eigen
    run_block = @timed begin
        (eigenvalues, eigenvectors) = eigen(cov(A), sortby = x -> -x)
    end
    
    (eigenvalues, eigenvectors) = run_block.value # Again
    elapsed_time = run_block.time
    mem_used = run_block.bytes / 1024^3 # Bytes to Gigabytes
    
    (training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used
end

setup_basics (generic function with 2 methods)

## Set folders to use, or random

In [ ]:
# Set folders to use, or random

# Get random folders 
#random_n_folders = get_random_n_folders(min_images_per_folder)

# Fixed
random_n_folders = [
 "./lfw/Guillermo_Coria/",
 "./lfw/Atal_Bihari_Vajpayee/",
 "./lfw/Kofi_Annan/",
 "./lfw/Nicole_Kidman/",
 "./lfw/Kofi_Annan/",
 "./lfw/Pervez_Musharraf/",
 "./lfw/Richard_Myers/",
 "./lfw/Tom_Daschle/",
 "./lfw/Amelie_Mauresmo/",
 "./lfw/Atal_Bihari_Vajpayee/"
];

# Reconstruction
## Measuring _data_ differences for: Float 16, 32, 64, for every k on every image

In [9]:
precisions = [Float16, Float32, Float64]

3-element Vector{DataType}:
 Float16
 Float32
 Float64

#### Setup:

In [ ]:
n_folders = 2
n_training_images = 4
n_testing_images = 4

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

#### [Auxiliary functions]:

In [ ]:
#
# Returns a VECTOR with the norm error between original and reconstructed using k vectors.
#
#           |  k=1    |    k=2   |   ...   |  k = index_X_percent |
#-----------------------------------------------------------------|
#   Image   |  error  |   error  |  ...    |        error         |

function reconstruct_for_k_eigenfaces(U, A, μ, training_images, display_images)
    # Projections
    P = transpose(U) * transpose(A)

    # Reconstruction using k eigenvectors
    reconstructed_images = (P' * U').+μ
    
    if display_images == true
        display(
            [
                Gray.(reshape(training_images[2,:], image_size, image_size)), # Original
                Gray.(reshape(reconstructed_images[2,:], image_size, image_size))  # Using k eigenfaces
            ]
        )
    end
    return norm.(eachrow(training_images - reconstructed_images), 2) # row_norms
end


# For every precision creates a table with the different reconstruction errors for every k on every image.
# Example:

#           |  k=1    |    k=2   |   ...   |  k = index_X_percent |
#-----------------------------------------------------------------|
# Image_1   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------|
# Image_2   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------|
#   ...     |   ...   |    ...   |   ...   |        ...           |
#-----------------------------------------------------------------|
# Image_M   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------|


function reconstruc_all_for_X_percent_of_k(percent)
    errors_by_precision = []
    
    # Random
    training_set = get_random_n_folders(min_images_per_folder)
    
    for used_precision in precisions
        
        # Getting the basics
        ((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set);
        
        # Using the to XX% most significant of all eigenvalues
        cumulative_eigenvalues = cumsum(eigenvalues./sum(eigenvalues))
        index_X_percent = findfirst(cumulative_eigenvalues .>= (percent/100))
        
        errors_by_k = zeros(Float64, n_folders*n_training_images, index_X_percent)
        for k in 1:index_X_percent
            errors_by_k[:,k] = reconstruct_for_k_eigenfaces(eigenvectors[:, 1:k], A, μ, training_images, false)
        end
            
        push!(errors_by_precision, errors_by_k)
    end
    errors_by_precision
end



#### Run function

In [ ]:
errors_by_precision = reconstruc_all_for_X_percent_of_k(99);

#### Analysis

In [ ]:
# Working with eps 64
epsilon = eps(Float64)

# Absolite diff bigger than eps? (64 vs 16)
result = abs.(errors_by_precision[3] - errors_by_precision[2]) .> epsilon

# Results
println(result)

## Measuring _mem and time_ differences for: Float 16, 32, 64 on setup

#### Run block

In [ ]:
array_n_folders[end]

In [ ]:
#
# Memory and time used to calculate covariance matrix and eigen for F16, 32, 64
#
mem_time_by_precision = []
    
#
# Original values -> Aux
aux_n_folders = n_folders
aux_n_training_images = n_training_images
aux_n_testing_images = n_testing_images
aux_random_n_folders = random_n_folders

# Different values to test:
array_n_folders = [2, 4] # <-------- Modify this
array_n_training_images = [4, 8] # <-------- Modify this
n_testing_images = 4

# Update {min_images_per_folder} using the bigger values
min_images_per_folder = n_testing_images + array_n_training_images[end] + 1; # +1 because google drive creating desktop.ini

# Use the new {min_images_per_folder} to get the folders
training_set = get_random_n_folders(min_images_per_folder)

for used_precision in precisions

    data = zeros(Float64, size(array_n_folders,1)*size(array_n_training_images,1), 4) 
    index = 1
    # Table data:

    # n_folders | n_training_images | elapsed_time | mem_used
    #    ..              ..                ..           ..
    #    ..              ..                ..           ..

    for aux1 in array_n_folders
        n_folders = aux1

        for aux2 in array_n_training_images
            n_training_images = aux2

            # Working here...
            ((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set[1:n_folders]);

            # Save data:
            data[index,:] = [n_folders, n_training_images, elapsed_time, mem_used]
            index += 1            
        end
    end

    push!(mem_time_by_precision, data)
end

# Aux -> Original values
n_folders = aux_n_folders
n_training_images = aux_n_training_images
n_testing_images = aux_n_testing_images;

In [ ]:
[]

#### Show data

In [ ]:
# Encabezado de la tabla
println("Float16:")
println("n_folders | n_training_images | elapsed_time | mem_used")
mem_time_by_precision[1]

In [ ]:
println("Float32:")
mem_time_by_precision[2]

In [ ]:
println("Float64:")
mem_time_by_precision[3]

## Plot eigenvalues

#### Setup data first:

In [ ]:
used_precision = Float64

n_folders = 5
n_training_images = 7
n_testing_images = 6

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision);

X_percent = 95
X_percent = X_percent / 100

#### Run block:

In [ ]:
cumulative_eigenvalues = cumsum(eigenvalues ./ sum(eigenvalues))
index_X_percent = findfirst(cumulative_eigenvalues .>= X_percent)

# To put X_percent in the middle
x_range = 1:index_X_percent * 2

# Plot 2 curves con el rango x limitado
plot(
    [eigenvalues[x_range] ./ sum(eigenvalues)], 
    label="eigenvalues",
    title="$(X_percent)% coverage at the $index_X_percent-th elem of $(size(eigenvalues, 1)) (~$(round(index_X_percent/(size(eigenvalues, 1)), digits=4))%)",
    legend=:outerbottom,
    titlefontsize=12
)

# Plot cumulative eigenvalues con el rango x limitado
plot!(cumulative_eigenvalues[x_range], label="cumulative eigenvalues")

# Add X% text
annotate!([(index_X_percent, cumulative_eigenvalues[index_X_percent], text("$(X_percent)%", 8))])

# Add x line
vline!([index_X_percent], line=:black, linewidth=0.5, label=nothing)

# Add y line
hline!([cumulative_eigenvalues[index_X_percent]], line=:black, linewidth=0.5, label=nothing)

# Recognition

In [ ]:
#show_training_images(training_images)

In [ ]:
#show_testing_images(testing_images)

#### Setup data first:

In [23]:
n_folders = 4
n_training_images = 16
n_testing_images = 6

min_images_per_folder = n_training_images + n_testing_images + 1; # +1 because google drive creating desktop.ini

# Selecting the top {X_percent}% of the eigenvalues
X_percent = 99
X_percent = X_percent / 100

0.99

In [24]:
# Check if there are as many folder after the filter using: min_images_per_folder
println("We want $n_folders folders with $(min_images_per_folder-1) min_images_per_folder, we have: $(size(get_useful_folders(min_images_per_folder), 1))")

We want 4 folders with 22 min_images_per_folder, we have: 53


#### Aux functions:

In [25]:
# Use k components to test accuracy.

function test_for_k_components(A, eigenfaces, random_n_folders, display_data)
    P = transpose(eigenfaces)*transpose(A);

    correct = 0.0
    all_min_dif = [] # We collect all the min norm diff
    for w in 1:size(testing_images,1)
        x_i = testing_images[w,:]' - μ
        P_i = transpose(eigenfaces) * transpose(x_i);

        dist=[]
        for j in 1:size(P,2)
            push!(dist, norm(P[:,j] - P_i))
        end
        pred = argmin(dist)
        push!(all_min_dif, dist[pred])

        celeb_tested = random_n_folders[ div(w - 1, n_testing_images) + 1 ]            
        celeb_predicted = random_n_folders[ div(pred - 1, n_training_images) + 1 ]
        
        if (celeb_tested == celeb_predicted)
            correct+=1
        end   
                
        if display_data == true
            println("_________________________________")
            println("Current: $celeb_tested (#$w)")
            println("Pred:    $celeb_predicted (#$pred)")

            println("Correct: $(correct/w*100)% ")

            IJulia.display([
                Gray.(reshape(testing_images[w,:], image_size, image_size)),
                Gray.(reshape(training_images[pred,:], image_size, image_size))
            ])
        end
    end
            
    accurracy = correct/size(testing_images,1)*100
    (accurracy, vec(all_min_dif))
end


test_for_k_components (generic function with 1 method)

#### Run block:

In [29]:
# The training set (USE THE SAME FOR EVERY PRECISION)
training_set = get_random_n_folders(min_images_per_folder)

# Firts run, to get eigenvalues and then -> index_X_percent
used_precision = Float16
((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set);

# Testing for the {X_percent}%
cumulative_eigenvalues = cumsum(eigenvalues ./ sum(eigenvalues))
index_X_percent = findfirst(cumulative_eigenvalues .>= X_percent)


# Table format for {accuracy_by_precision}
#
#           |  Float16  |  Float32  |  Float64  |
#-----------|-----------|-----------|-----------|
#    k=1    | correct%  | correct%  | correct%  |
#    k=2    | correct%  | correct%  | correct%  |
#    ...    |   ...     |   ...     |   ...     |
#   k = X%  | correct%  | correct%  | correct%  |


# Table format for {recognition_dif_by_precision}
#
#                |  k=1    |    k=2   |   ...   |  k = index_X_percent |
#----------------------------------------------------------------------|
# Test_image_1   |  error  |   error  |  ...    |        error         |
#----------------------------------------------------------------------|
# Test_image_2   |  error  |   error  |  ...    |        error         |
#----------------------------------------------------------------------|
#        ...     |   ...   |    ...   |   ...   |        ...           |
#----------------------------------------------------------------------|
# Test_image_M   |  error  |   error  |  ...    |        error         |
#-----------------------------------------------------------------     |


# The data:
#
accuracy_by_precision = zeros(Float64, index_X_percent, 3)
recognition_dif_by_precision = []

# TODO: Iterations?

for (col, used_precision) in enumerate(precisions)
    ((training_images, testing_images), μ, A, eigenvalues, eigenvectors, elapsed_time, mem_used) = setup_basics(used_precision, training_set);

    recognition_diff = zeros(Float64, n_folders*n_testing_images, index_X_percent)
    
    for k in 1:index_X_percent

        # accuracy = A single value
        # diff     = A complete column
        (accuracy, diff) = test_for_k_components(A, eigenvectors[:,1:k], training_set, false)

        accuracy_by_precision[k,col] = accuracy
        recognition_diff[:,k] = diff
        
    end
    
    push!(recognition_dif_by_precision, recognition_diff)
end

In [30]:
accuracy_by_precision

55×3 Matrix{Float64}:
 25.0     25.0     25.0
 50.0     50.0     50.0
 50.0     50.0     50.0
 50.0     50.0     50.0
 45.8333  45.8333  45.8333
 50.0     50.0     50.0
 58.3333  58.3333  58.3333
 62.5     62.5     62.5
 70.8333  70.8333  70.8333
 66.6667  66.6667  66.6667
 66.6667  66.6667  66.6667
 66.6667  66.6667  66.6667
 66.6667  66.6667  66.6667
  ⋮                
 62.5     62.5     62.5
 62.5     62.5     62.5
 62.5     62.5     62.5
 62.5     62.5     62.5
 62.5     62.5     62.5
 62.5     62.5     62.5
 62.5     62.5     62.5
 62.5     62.5     62.5
 66.6667  66.6667  66.6667
 66.6667  66.6667  66.6667
 66.6667  66.6667  66.6667
 66.6667  66.6667  66.6667

In [31]:
recognition_dif_by_precision[3]

24×55 Matrix{Float64}:
 0.208648   0.739821  3.20446   3.21128  …  10.7832   10.8401   10.8417
 0.146082   0.790647  1.49252   1.81987     10.7736   10.7764   10.7843
 0.0194562  2.30985   3.17064   3.49171     14.1605   14.2546   14.2564
 0.0230539  1.16578   5.33679   5.99195     14.0265   14.0375   14.0377
 0.113374   1.15541   3.08742   4.62805     12.1702   12.1727   12.2282
 0.306637   2.61853   4.51483   5.14994  …  13.0097   13.0835   13.2247
 0.370003   1.86477   3.03135   3.11599     11.6976   11.6977   11.6989
 0.569699   1.11394   1.3978    1.98023     10.5192   10.5589   10.5772
 1.123      1.66049   2.20035   3.15844      8.20307   8.28277   8.29033
 0.0516705  0.56199   0.665732  1.42374     11.6646   11.7346   11.8444
 1.25304    1.53345   4.24222   5.85566  …  16.0629   16.314    16.3461
 1.5972     2.31413   2.51018   4.30703     13.1344   13.2209   13.222
 0.154884   2.73505   4.84932   6.28002     13.7212   13.732    13.7665
 0.079181   1.14457   3.63452   4.13079  

In [32]:
recognition_dif_by_precision[2]

24×55 Matrix{Float64}:
 0.208643   0.73982   3.20446   3.21128  …  10.7832   10.8401   10.8417
 0.146081   0.790646  1.49251   1.81987     10.7736   10.7764   10.7843
 0.0194566  2.30985   3.17065   3.49172     14.1605   14.2546   14.2564
 0.0230474  1.16578   5.33679   5.99194     14.0265   14.0375   14.0377
 0.113379   1.15541   3.08743   4.62805     12.1702   12.1727   12.2282
 0.306635   2.61853   4.51484   5.14994  …  13.0097   13.0835   13.2247
 0.370003   1.86477   3.03135   3.11599     11.6976   11.6977   11.6989
 0.569703   1.11393   1.3978    1.98024     10.5192   10.5589   10.5772
 1.123      1.66049   2.20035   3.15844      8.20307   8.28277   8.29034
 0.051671   0.56199   0.665737  1.42374     11.6646   11.7346   11.8444
 1.25304    1.53345   4.24221   5.85565  …  16.0629   16.314    16.3461
 1.59719    2.31412   2.51018   4.30704     13.1344   13.2209   13.222
 0.154885   2.73505   4.84931   6.28002     13.7212   13.732    13.7665
 0.0791874  1.14457   3.63452   4.13079  

In [33]:
recognition_dif_by_precision[1]

24×55 Matrix{Float64}:
 0.226562   0.73877   3.20703   3.21484  …  10.7812   10.8359   10.8359
 0.152344   0.792969  1.49316   1.81543     10.7734   10.7812   10.7891
 0.0234375  2.31055   3.17188   3.49219     14.1641   14.2578   14.2578
 0.015625   1.16992   5.34766   5.99609     14.0312   14.0391   14.0391
 0.113281   1.14453   3.08594   4.62891     12.1719   12.1719   12.2266
 0.300781   2.62891   4.51172   5.14453  …  13.0      13.0781   13.2188
 0.370117   1.86914   3.02734   3.11133     11.6953   11.6953   11.6953
 0.5625     1.11621   1.40039   1.98535     10.5156   10.5547   10.5781
 1.11719    1.6582    2.20117   3.16602      8.21094   8.28906   8.29688
 0.046875   0.563965  0.664551  1.43457     11.6641   11.7344   11.8516
 1.21875    1.50684   4.23438   5.84766  …  16.0625   16.3125   16.3438
 1.60938    2.30664   2.50195   4.30078     13.1328   13.2188   13.2188
 0.164062   2.73633   4.85156   6.26953     13.7266   13.7344   13.7734
 0.0898438  1.14844   3.63867   4.13672 

#### Comparison

In [39]:
# Working with eps 64
epsilon = eps(Float64)

# Abs diff bigger than eps?
# [3] = 64
# [2] = 32
# [1] = 16
result = abs.(recognition_dif_by_precision[3] - recognition_dif_by_precision[1]) .> epsilon

# Results
println("1 = Difference between values is > than epsilon $(typeof(epsilon))")
println("0 = No\n")
println(result)

1 = Difference between values is > than epsilon Float64
0 = No

Bool[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1; 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [ ]:
"""
DATA FRAMES ---> EXCEL

# Define DF
df = DataFrame(
    used_precision = UInt8[],
    image_size = UInt16[],
    n_folders = UInt16[],
    n_training_images = UInt8[],
    n_testing_images = UInt8[],
    eigen_elapsed_time = Float64[],
    used_eigenfaces = UInt8[],
    norm_error = Float64[],
    acurracy = Float16[]
)

# Define a row
row_dict = Dict{String, Any}()

# Save data
row_dict["used_precision"] = p
row_dict["image_size"] = image_size
row_dict["persons"] = used_persons
row_dict["images_db"] = images_to_db
row_dict["images_to_test"] = images_to_test
row_dict["covariance_calc_time"] = covariance_elapsed_time
row_dict["eigen_elapsed_time"] = eigen_elapsed_time
row_dict["used_eigenfaces"] = n_components
row_dict["norm_error"] = error
row_dict["acurracy"] = acurracy

# Push to DF
push!(df, row_dict)

# Write excel
XLSX.writetable("eigenframe.xlsx", df, overwrite=true, sheetname="Recognition")
""";